In [1]:
import torch.nn as nn
import torch
import random
import os
import pandas as pd

<h6> In this notebook, we will try to unjumble a sentence using Encoder-Decoder Architecture built using <br><br>
 Recurrent Networks like GRU, LSTM and Bi-directional LSTMs.</h6>
<h6> The Data is located here: ../../Datasets/Jumble_Unjumble/ </h6>

#### Read Data

In [2]:
# train_df = pd.read_csv("../../Datasets/Jumble_Unjumble/Train_400.tsv",sep="\t")
# test_df = pd.read_csv("../../Datasets/Jumble_Unjumble/Test_100.tsv",sep="\t")
# print(train_df.shape, test_df.shape)
# train_df.head()

In [3]:
jumbled_df = pd.read_csv("../../Datasets/Jumble_Unjumble/processed_jumbled.txt",sep="\t",header=None)
unjumbled_df = pd.read_csv("../../Datasets/Jumble_Unjumble/processed_unjumbled.txt",sep="\t",header=None)
jumbled_df.columns = ["jumbled_sentences"]
unjumbled_df.columns = ["unjumbled_sentences"]
df = pd.concat([jumbled_df,unjumbled_df],axis=1)
train_df = df.sample(frac=0.8, random_state=42) 
test_df = df.drop(train_df.index)
print(train_df.shape, test_df.shape)
train_df.head()

(32368, 2) (8092, 2)


,jumbled_sentences,unjumbled_sentences
32760,tools and a man gardening inside two holding a...,a man and two women are inside a greenhouse ho...
31413,meandering at people of the walkway stand . up...,people stand at the bottom of a meandering wal...
4325,standing a rock . on man view the shorts a out...,a man in shorts is standing on a rock looking ...
28232,to a a in on shirt little red a holds pole nea...,a little girl in a red shirt holds on to a pol...
28438,children two in <unk> play the melting .,two children play in the melting <unk> .


#### Build Vocab using jumbled_sentences of Train + Test dataset. Ideally only Train dataset should be used.

In [4]:
class VocabBuilder:
    def __init__(self,text_corpus,unknown_token=None,pad_token=None,sos_token=None,eos_token=None):
        '''
        text_corpus = "This is first sentence. This is second sentence. This is another sentence"
        '''
        self.text_corpus = text_corpus
        self.unknown_token = unknown_token or "<unk>"
        self.pad_token = pad_token or "<pad>"
        self.sos_token = sos_token or "<sos>"
        self.eos_token = eos_token or "<eos>"
        self.word_to_index, self.index_to_word = self.get_vocabs()
                        
    def get_vocabs(self):
        word_to_index = {}
        index_count = 0
        all_unique_words = set(self.text_corpus.split(" "))
        for index, word in enumerate(all_unique_words):
            word_to_index[word] = index
        if self.pad_token not in word_to_index: word_to_index[self.pad_token] = index + 1
        if self.sos_token not in word_to_index: word_to_index[self.sos_token] = index + 2
        if self.eos_token not in word_to_index: word_to_index[self.eos_token] = index + 3
        if self.unknown_token not in word_to_index: word_to_index[self.unknown_token] = index + 4
        index_to_word = {v:k for k,v in word_to_index.items()}
        return word_to_index, index_to_word

In [5]:
text_corpus_1 = " ".join(train_df["jumbled_sentences"].tolist())
text_corpus_2 = " ".join(test_df["jumbled_sentences"].tolist())
text_corpus = text_corpus_1 + " " + text_corpus_2
print(text_corpus[:200])

tools and a man gardening inside two holding are . women a greenhouse  meandering at people of the walkway stand . uphill that bottom goes a  standing a rock . on man view the shorts a out from lookin


In [6]:
unknown_token = "<unk>"
pad_token = "<pad>"
sos_token = "<sos>"
eos_token = "<eos>"
vocab_builder = VocabBuilder(text_corpus,unknown_token,pad_token,sos_token,eos_token)
print("WordToIndex Dict length:",len(vocab_builder.word_to_index))
print("IndexToWord Dict length:",len(vocab_builder.index_to_word))

WordToIndex Dict length: 5242
IndexToWord Dict length: 5242


#### Create X_Encoder, X_Decoder and Y
X_encoder is the matrix of words in jumbled_sentences, each sentence suffixed by "eos" token <br>
X_decoder is the matrix of unjumbled_sentences, each sentence prefixed by "sos" token <br>
Y is the matrix of unjumbled_sentences, each sentence suffixed by "eos" token <br>

Do this for both train and test data

In [7]:
def get_Xe_Xd_Y(dataframe, sos_token, eos_token):
    jumbled_sentences = dataframe["jumbled_sentences"].tolist()
    unjumbled_sentences = dataframe["unjumbled_sentences"].tolist()
    X_encoder_words = [el.split(" ") + [eos_token] for el in jumbled_sentences]
    X_decoder_words = [[sos_token] + el.split(" ") for el in unjumbled_sentences]
    Y_words = [el.split(" ") + [eos_token] for el in unjumbled_sentences]
    return X_encoder_words, X_decoder_words, Y_words

X_encoder_words_tr, X_decoder_words_tr, Y_words_tr = get_Xe_Xd_Y(train_df, sos_token, eos_token)
X_encoder_words_test, X_decoder_words_test, Y_words_test = get_Xe_Xd_Y(test_df, sos_token, eos_token)
print("X Encoder train length:",len(X_encoder_words_tr))
print("X Encoder test length:",len(X_encoder_words_test))
print("Sample X_encoder_train:",X_encoder_words_tr[0])
print("Sample X_decoder_train:",X_decoder_words_tr[0])
print("Sample Y_train:",Y_words_tr[0])

X Encoder train length: 32368
X Encoder test length: 8092
Sample X_encoder_train: ['tools', 'and', 'a', 'man', 'gardening', 'inside', 'two', 'holding', 'are', '.', 'women', 'a', 'greenhouse', '', '<eos>']
Sample X_decoder_train: ['<sos>', 'a', 'man', 'and', 'two', 'women', 'are', 'inside', 'a', 'greenhouse', 'holding', 'gardening', 'tools', '.', '']
Sample Y_train: ['a', 'man', 'and', 'two', 'women', 'are', 'inside', 'a', 'greenhouse', 'holding', 'gardening', 'tools', '.', '', '<eos>']


#### Map X_encoder, X_decoder and Y using Vocab

In [8]:
class Word_Index_Mapper:
    def __init__(self,word_to_index,index_to_word, unknown_token):
        self.word_to_index = word_to_index
        self.index_to_word = index_to_word
        self.unknown_token = unknown_token
    
    def get_encoding(self,sentence):
        '''
        sentence must be a list of words.
        Ex: ["Climate","change","is","a","pressing","global","issue"]
        '''
        encoded_sentence = []
        for word in sentence:
            if word in self.word_to_index: encoded_sentence.append(self.word_to_index[word])
            else: encoded_sentence.append(self.word_to_index[self.unknown_token])
        return encoded_sentence
    
    def get_decoding(self,encoded_sentence):
        '''
        encoded_sentence must be a list of vocab indices.
        Ex: encoded_sentence = [24,21,4,1,..] 
        '''
        sentence = [self.index_to_word[index] for index in encoded_sentence]
        return " ".join(sentence)

In [9]:
def map_words_to_indices(word_index_mapper, max_sequence_length, word_matrix):
    index_matrix = []
    for el in word_matrix:
        el = el[:max_sequence_length]
        if len(el) < max_sequence_length:
            pad_tokens_to_append = max_sequence_length - len(el)
            el = el + [pad_token]*pad_tokens_to_append
        index_matrix.append(word_index_mapper.get_encoding(el))
    return index_matrix

In [10]:
max_sequence_length = 25
word_index_mapper = Word_Index_Mapper(vocab_builder.word_to_index, vocab_builder.index_to_word, unknown_token)
X_encoder_indices_tr = map_words_to_indices(word_index_mapper, max_sequence_length, X_encoder_words_tr)
X_decoder_indices_tr = map_words_to_indices(word_index_mapper, max_sequence_length, X_decoder_words_tr)
Y_indices_tr = map_words_to_indices(word_index_mapper, max_sequence_length, Y_words_tr)
X_encoder_indices_test = map_words_to_indices(word_index_mapper, max_sequence_length, X_encoder_words_test)
X_decoder_indices_test = map_words_to_indices(word_index_mapper, max_sequence_length, X_decoder_words_test)
Y_indices_test = map_words_to_indices(word_index_mapper, max_sequence_length, Y_words_test)
print("X Encoder train length:",len(X_encoder_indices_tr))
print("X Encoder test length:",len(X_encoder_indices_test))
print("Sample X_encoder_train:",X_encoder_indices_tr[0])
print("Sample X_decoder_train:",X_decoder_indices_tr[0])
print("Sample Y_train:",Y_indices_tr[0])

X Encoder train length: 32368
X Encoder test length: 8092
Sample X_encoder_train: [857, 3489, 4223, 4354, 1988, 2041, 3618, 58, 5033, 3732, 412, 4223, 2350, 0, 5241, 5239, 5239, 5239, 5239, 5239, 5239, 5239, 5239, 5239, 5239]
Sample X_decoder_train: [5240, 4223, 4354, 3489, 3618, 412, 5033, 2041, 4223, 2350, 58, 1988, 857, 3732, 0, 5239, 5239, 5239, 5239, 5239, 5239, 5239, 5239, 5239, 5239]
Sample Y_train: [4223, 4354, 3489, 3618, 412, 5033, 2041, 4223, 2350, 58, 1988, 857, 3732, 0, 5241, 5239, 5239, 5239, 5239, 5239, 5239, 5239, 5239, 5239, 5239]


#### Unknown token statistics - must be 0, as both Train and Test dataset has been used for vocab creation

In [11]:
X_test_temp = []
unknown_token_counts = 0
for el in X_decoder_indices_test:
    temp_list = word_index_mapper.get_decoding(el)
    unknown_token_counts += temp_list.count(unknown_token)
    X_test_temp.append(temp_list)
print(unknown_token_counts)
print(X_test_temp[4])

741
<sos> a small child grips onto the red ropes at the playground .  <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>


### Define Model

In [12]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, dropout, debug):
        super(Encoder, self).__init__()
        self.debug = debug
        self.hid_dim = hid_dim
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.gru = nn.GRU(emb_dim, hid_dim, batch_first=True, bidirectional = True)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        embedded = self.embedding(x)
        dropped_out = self.dropout(embedded) 
        output, hidden = self.gru(dropped_out)
        hidden_concatenated = torch.cat((hidden[-1],hidden[-2]),axis=1)
        hidden_unsqueezed = hidden_concatenated.unsqueeze(0)
        if self.debug: 
            print("-----------Encoder----------:")
            print("Input Data shape:",x.shape)
            print("After Embedding Layer:",embedded.shape)
            print("After Dropout Layer:",dropped_out.shape)
            print("Outputs and hidden shape from GRU:",output.shape, hidden.shape)
            print("Concatenated hidden shape:", hidden_concatenated.shape)
            print("Unsqueezed hidden shape:", hidden_unsqueezed.shape)
        return output, hidden_unsqueezed

class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, debug):
        super(Decoder, self).__init__()
        self.debug = debug
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.gru = nn.GRU(emb_dim, hid_dim,  batch_first=True)
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
    def forward(self, x, encoder_hidden):
        embedded = self.embedding(x)
        output, hidden = self.gru(embedded, encoder_hidden)
        reshaped_output = output.reshape(-1,output.shape[2])
        prediction = self.fc_out(reshaped_output)
        if self.debug: 
            print("-----------Decoder----------:")
            print("Input Data shape, X:",x.shape, ", Encoder hidden state:", encoder_hidden.shape)
            print("After Embedding Layer:",embedded.shape)
            print("Outputs and hidden shape from GRU:",output.shape, hidden.shape)
            print("Reshaped Output:", reshaped_output.shape)
            print("After FC layer:", prediction.shape)
        return prediction, hidden
    

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super(Seq2Seq, self).__init__() 
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
    
    def forward(self, encoder_input, decoder_input, teacher_forcing_ratio=0.5): 
        encoder_outputs, encoder_hidden = self.encoder(encoder_input) 
        outputs, _ = self.decoder(decoder_input,encoder_hidden)
        return outputs

In [13]:
def predict(model, X_encoder_indices_test, X_decoder_indices_test, 
            X_encoder_words_test, X_decoder_words_test,
            word_index_mapper, device):
    data_index = random.randint(0,100)
    Xe_b = torch.tensor([X_encoder_indices_test[data_index]]).to(device)
    print(data_index)
    print(X_encoder_words_test[data_index])
    print(X_decoder_words_test[data_index])
    
    model.eval()
    with torch.no_grad():
        ht,ht_for_decoder = model.encoder(Xe_b)
        sos_word = torch.tensor([[word_index_mapper.word_to_index["<sos>"]]]).to(device)
        op,ht = model.decoder(sos_word,ht_for_decoder)
        unjumbled_sentence = []
        for i in range(25):
            predicted_word = torch.argmax(op,axis=1).tolist()
#             print("Predicted .....................",predicted_word)
            if predicted_word[0] == word_index_mapper.word_to_index["<eos>"]: break
            unjumbled_sentence.append(word_index_mapper.index_to_word[predicted_word[0]])
            op,ht = model.decoder(torch.tensor([predicted_word]).to(device),ht)
        print("_______________________________________")
        print(unjumbled_sentence)

### Sample Training And Prediction

In [16]:
INPUT_DIM = len(word_index_mapper.word_to_index) # Size of source vocabulary 
OUTPUT_DIM = len(word_index_mapper.word_to_index) # Size of target vocabulary 
ENC_EMB_DIM = 256 
DEC_EMB_DIM = 256 
HID_DIM = 512 
N_LAYERS = 2 
ENC_DROPOUT = 0.5 
DEC_DROPOUT = 0.5
device = "cpu"
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, ENC_DROPOUT, debug = True) 
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM*2, debug = True) 
model = Seq2Seq(enc, dec, device).to(device)
optimizer = torch.optim.Adam(model.parameters()) 
criterion = nn.CrossEntropyLoss(ignore_index=word_index_mapper.word_to_index["<pad>"])

In [17]:
data_index = 6
batch_size = 5
model.train()

optimizer.zero_grad()
Xe_b = torch.tensor(X_encoder_indices_tr[data_index:data_index+batch_size]).to(device)
Xd_b = torch.tensor(X_decoder_indices_tr[data_index:data_index+batch_size]).to(device)
Y_b = torch.tensor(Y_indices_tr[data_index:data_index+batch_size]).to(device)
print(Xe_b.shape,Xd_b.shape,Y_b.shape)

output = model(Xe_b, Xd_b)
loss = criterion(output, Y_b.view(-1))
loss.backward()
optimizer.step()

print("---------------------------------------------")
optimizer.zero_grad()
Xe_b = torch.tensor(X_encoder_indices_tr[data_index+1:data_index+batch_size+1]).to(device)
Xd_b = torch.tensor(X_decoder_indices_tr[data_index+1:data_index+batch_size+1]).to(device)
Y_b = torch.tensor(Y_indices_tr[data_index+1:data_index+batch_size+1]).to(device)
print(Xe_b.shape,Xd_b.shape,Y_b.shape)

output = model(Xe_b, Xd_b)
loss = criterion(output, Y_b.view(-1))
loss.backward()
optimizer.step()

torch.Size([5, 25]) torch.Size([5, 25]) torch.Size([5, 25])
-----------Encoder----------:
Input Data shape: torch.Size([5, 25])
After Embedding Layer: torch.Size([5, 25, 256])
After Dropout Layer: torch.Size([5, 25, 256])
Outputs and hidden shape from GRU: torch.Size([5, 25, 1024]) torch.Size([2, 5, 512])
Concatenated hidden shape: torch.Size([5, 1024])
Unsqueezed hidden shape: torch.Size([1, 5, 1024])
-----------Decoder----------:
Input Data shape, X: torch.Size([5, 25]) , Encoder hidden state: torch.Size([1, 5, 1024])
After Embedding Layer: torch.Size([5, 25, 256])
Outputs and hidden shape from GRU: torch.Size([5, 25, 1024]) torch.Size([1, 5, 1024])
Reshaped Output: torch.Size([125, 1024])
After FC layer: torch.Size([125, 5242])
---------------------------------------------
torch.Size([5, 25]) torch.Size([5, 25]) torch.Size([5, 25])
-----------Encoder----------:
Input Data shape: torch.Size([5, 25])
After Embedding Layer: torch.Size([5, 25, 256])
After Dropout Layer: torch.Size([5, 2

In [18]:
predict(model,X_encoder_indices_test,X_decoder_indices_test, X_encoder_words_test, X_decoder_words_test, word_index_mapper, device)

68
['a', 'jumping', 'a', 'water', 'boy', 'on', '<unk>', '', '<eos>']
['<sos>', 'a', 'boy', 'jumping', 'on', 'a', 'water', '<unk>', '']
-----------Encoder----------:
Input Data shape: torch.Size([1, 25])
After Embedding Layer: torch.Size([1, 25, 256])
After Dropout Layer: torch.Size([1, 25, 256])
Outputs and hidden shape from GRU: torch.Size([1, 25, 1024]) torch.Size([2, 1, 512])
Concatenated hidden shape: torch.Size([1, 1024])
Unsqueezed hidden shape: torch.Size([1, 1, 1024])
-----------Decoder----------:
Input Data shape, X: torch.Size([1, 1]) , Encoder hidden state: torch.Size([1, 1, 1024])
After Embedding Layer: torch.Size([1, 1, 256])
Outputs and hidden shape from GRU: torch.Size([1, 1, 1024]) torch.Size([1, 1, 1024])
Reshaped Output: torch.Size([1, 1024])
After FC layer: torch.Size([1, 5242])
-----------Decoder----------:
Input Data shape, X: torch.Size([1, 1]) , Encoder hidden state: torch.Size([1, 1, 1024])
After Embedding Layer: torch.Size([1, 1, 256])
Outputs and hidden shape 

## For Actual Training

In [21]:
device = "cpu" #torch.device("cuda:0")
batch_size = 50
INPUT_DIM = len(word_index_mapper.word_to_index) # Size of source vocabulary 
OUTPUT_DIM = len(word_index_mapper.word_to_index) # Size of target vocabulary 
ENC_EMB_DIM = 128 
DEC_EMB_DIM = 128 
HID_DIM = 250 
N_LAYERS = 2 
ENC_DROPOUT = 0.5 
DEC_DROPOUT = 0.5
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, ENC_DROPOUT, debug = False) 
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM*2, debug = False) 
model = Seq2Seq(enc, dec, device).to(device)
optimizer = torch.optim.Adam(model.parameters()) 
criterion = nn.CrossEntropyLoss(ignore_index=word_index_mapper.word_to_index["<pad>"])
epochs = 50

In [23]:
epochs = 5

In [24]:
for i in range(epochs):
#     init_ht_for_encoder = model_encoder.init_hidden().to(device)
    model.train()
    epoch_loss = 0
    for j in range(0,len(X_encoder_indices_tr),batch_size):
        optimizer.zero_grad()
        Xe_b = torch.tensor(X_encoder_indices_tr[j:j+batch_size]).to(device)
        Xd_b = torch.tensor(X_decoder_indices_tr[j:j+batch_size]).to(device)
        Y_b = torch.tensor(Y_indices_tr[j:j+batch_size]).to(device)
        op = model(Xe_b,Xd_b)
        loss = criterion(op,Y_b.reshape(-1))
        loss.backward()
        optimizer.step()
        batch_loss = loss.item()
        epoch_loss += batch_loss
        if j%1000 == 0: print("Epoch:",i,"Batch:",j,"Loss:",batch_loss)
    print("______________________________________")
    print("Epoch Loss:",epoch_loss)
    predict(model,X_encoder_indices_test,X_decoder_indices_test, 
            X_encoder_words_test, X_decoder_words_test, word_index_mapper, device)
    print("_______________________________________")

Epoch: 0 Batch: 0 Loss: 0.06488276273012161
Epoch: 0 Batch: 1000 Loss: 0.08332639932632446
Epoch: 0 Batch: 2000 Loss: 0.09826172143220901
Epoch: 0 Batch: 3000 Loss: 0.0484938807785511
Epoch: 0 Batch: 4000 Loss: 0.0832718014717102
Epoch: 0 Batch: 5000 Loss: 0.05295179411768913
Epoch: 0 Batch: 6000 Loss: 0.09563960134983063
Epoch: 0 Batch: 7000 Loss: 0.05169978365302086
Epoch: 0 Batch: 8000 Loss: 0.07165710628032684
Epoch: 0 Batch: 9000 Loss: 0.07759127020835876
Epoch: 0 Batch: 10000 Loss: 0.07503162324428558
Epoch: 0 Batch: 11000 Loss: 0.044686101377010345
Epoch: 0 Batch: 12000 Loss: 0.08322243392467499
Epoch: 0 Batch: 13000 Loss: 0.07350403070449829
Epoch: 0 Batch: 14000 Loss: 0.06829915940761566
Epoch: 0 Batch: 15000 Loss: 0.06235787272453308
Epoch: 0 Batch: 16000 Loss: 0.06498934328556061
Epoch: 0 Batch: 17000 Loss: 0.055059801787137985
Epoch: 0 Batch: 18000 Loss: 0.08079463243484497
Epoch: 0 Batch: 19000 Loss: 0.08060895651578903
Epoch: 0 Batch: 20000 Loss: 0.06775505840778351
Epoch

Epoch: 4 Batch: 1000 Loss: 0.08501055091619492
Epoch: 4 Batch: 2000 Loss: 0.05777866393327713
Epoch: 4 Batch: 3000 Loss: 0.0744839534163475
Epoch: 4 Batch: 4000 Loss: 0.07781413942575455
Epoch: 4 Batch: 5000 Loss: 0.07074024528265
Epoch: 4 Batch: 6000 Loss: 0.07213260233402252
Epoch: 4 Batch: 7000 Loss: 0.0653424859046936
Epoch: 4 Batch: 8000 Loss: 0.04089247062802315
Epoch: 4 Batch: 9000 Loss: 0.07852138578891754
Epoch: 4 Batch: 10000 Loss: 0.10024943202733994
Epoch: 4 Batch: 11000 Loss: 0.056304190307855606
Epoch: 4 Batch: 12000 Loss: 0.0796826034784317
Epoch: 4 Batch: 13000 Loss: 0.04819009453058243
Epoch: 4 Batch: 14000 Loss: 0.07925969362258911
Epoch: 4 Batch: 15000 Loss: 0.060639213770627975
Epoch: 4 Batch: 16000 Loss: 0.05457226559519768
Epoch: 4 Batch: 17000 Loss: 0.05291018262505531
Epoch: 4 Batch: 18000 Loss: 0.06610732525587082
Epoch: 4 Batch: 19000 Loss: 0.07297436147928238
Epoch: 4 Batch: 20000 Loss: 0.06230543926358223
Epoch: 4 Batch: 21000 Loss: 0.06230843439698219
Epoch

In [25]:
a = torch.ones(5,10,20)
a.shape

torch.Size([5, 10, 20])

In [35]:
b = a[-2]
b.shape

torch.Size([10, 20])

In [34]:
c = b.unsqueeze(dim=0)
c.shape

torch.Size([1, 10, 20])

In [44]:
d = a.reshape(-1,a.shape[2])
d.shape

torch.Size([50, 20])

In [50]:
e = torch.ones(8,5242)
e.shape

torch.Size([8, 5242])

In [51]:
f = torch.argmax(e,axis=1)
f.shape, f, f.tolist()

(torch.Size([8]), tensor([0, 0, 0, 0, 0, 0, 0, 0]), [0, 0, 0, 0, 0, 0, 0, 0])

In [49]:
g = torch.softmax(e,axis=1)
g.shape

torch.Size([1, 5242])